In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
from keras.optimizers import Adam
import os
from IPython.display import display
import json
from os.path import exists
from keras.applications import EfficientNetV2B1

from numpy.random import choice as choose

import sys
sys.path.insert(0, "../Src/")
import basicUtil
import transferUtil

%autosave 5

Autosaving every 5 seconds


In [2]:
def createTransferableModelWithTwoDenseLayers(base_model, numberOfNeuronsPerFCLayer, adamLearningRate, L2Rate):
    model = keras.Sequential()
    model.add(base_model)
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(numberOfNeuronsPerFCLayer,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(L2Rate)))
    model.add(keras.layers.Dense(numberOfNeuronsPerFCLayer,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(L2Rate)))
    model.add(keras.layers.Dense(1,activation='sigmoid'))
            
    adamOptimizer = keras.optimizers.legacy.Adam(learning_rate=adamLearningRate)       
    model.compile(optimizer=adamOptimizer,loss='binary_crossentropy', metrics=basicUtil.f1_score)
    return model

In [3]:
def increaseIntegerRange(argumentList):
    newMinimum = int(np.min(argumentList) / 2)
    newMinimum = np.max([newMinimum],1)
    newMaximum = int(np.max(argumentList) * 2)
    return range(newMinimum,newMaximum)

In [4]:
def increaseFloatRange(argumentTuple):
    newMinimum = argumentTuple[0] / 2
    newMaximum = argumentTuple[1] * 2
    return (newMinimum,newMaximum)

In [5]:
imageShape = (180,180)
inputShape = [imageShape[0],imageShape[1],3]
train = transferUtil.loadDataWithResizeShape("training",imageShape)
dev,test = transferUtil.loadDataWithResizeShape("testing",imageShape,.5)

base_model = EfficientNetV2B1(
    include_top=False,
    classes=len(train.class_names),
    classifier_activation=None,
    include_preprocessing=False,
    input_shape=inputShape
)
for layer in base_model.layers:
    layer.trainable = False

possibleNeuronsPerLayer = basicUtil.createRangeFromMidpoint(10,10)
possibleEpochs = basicUtil.createRangeFromMidpoint(18,18)

adamLearningRateCriticalPoints = (0.000333/2,0.000333*2)
L2CriticalPoints = (29.89698/2,29.89698*2) 

n_NeuronsPerFCLayers = []
n_Epochs = []
adamLearningRates = []
L2Rates = []
trainScores = []
devScores = []

trial = 0
bestDevScore = 0
oversizedNeuronNumbers = []
oversizedEpochNumbers = []

while trial < 100:
    numberOfNeuronsPerFCLayer = choose(possibleNeuronsPerLayer)
    numberOfEpochs = choose(possibleEpochs)
    
    adamLearningRate = basicUtil.generateAdamLearningRate(adamLearningRateCriticalPoints[0],adamLearningRateCriticalPoints[1])
    L2Rate = basicUtil.generateL2(L2CriticalPoints[0],L2CriticalPoints[1])
    model = createTransferableModelWithTwoDenseLayers(base_model, numberOfNeuronsPerFCLayer, adamLearningRate, L2Rate)
    
    model.fit(train,epochs=numberOfEpochs,verbose=0)

    model_path = f'../Models/EffecientNetTrials/effecientnet_model_{trial}.h5'
    model.save(model_path)
    model_size = os.path.getsize(model_path) / (1024 * 1024)
    if model_size < 40:
        oversizedNeuronNumbers = []
        oversizedEpochNumbers = []
        print('trainScore')
        trainScore = model.evaluate(train)[1]
        print('devScore')
        devScore = model.evaluate(dev)[1]

        if devScore > bestDevScore:
            testScore = model.evaluate(test)[1]
            model_path = f'../Models/best_effecientnet_model_.h5'
            model.save(model_path)
            bestModelParams = {
                'n_NeuronsPerFCLayers' : int(numberOfNeuronsPerFCLayer),
                'n_Epochs' : int(numberOfEpochs),
                'adamLearningRates' : adamLearningRate,
                'L2Rates' : L2Rate,
                'modelSize' : model_size,
                'trainScore': trainScore,
                'devScore': devScore,
                'testScore': testScore
            }
            with open('../Models/best_effecientnet_model_params.json', 'w') as f:
                json.dump(bestModelParams, f)
            bestDevScore = devScore       

        n_NeuronsPerFCLayers.append(numberOfNeuronsPerFCLayer)
        n_Epochs.append(numberOfEpochs)
        
        adamLearningRates.append(adamLearningRate)
        L2Rates.append(L2Rate)
        trainScores.append(trainScore)
        devScores.append(devScore)
        
        print('concluding trial ',trial)
        trial += 1
    else:
        print(f'redoing trial {trial}. Model was {model_size}MB.')
        failedTrial = transferUtil.createTransferModelParametersDF([numberOfNeuronsPerFCLayer],[numberOfEpochs],[adamLearningRate],
                                                                   [L2Rate],[np.nan],[np.nan])
        display(failedTrial)
        oversizedNeuronNumbers.append(numberOfNeuronsPerFCLayer)
        oversizedEpochNumbers.append(numberOfEpochs)
        if len(oversizedNeuronNumbers) >= 3:
            possibleNeuronsPerLayer = [x for x in possibleNeuronsPerLayer if x < np.mean(oversizedNeuronNumbers)]
            possibleEpochs = [x for x in possibleEpochs if x < np.mean(oversizedEpochNumbers)]
            oversizedNeuronNumbers = []
            oversizedEpochNumbers = []
        
    if (trial % 10 == 9): 
        modelParametersDF = transferUtil.createTransferModelParametersDF(n_NeuronsPerFCLayers,n_Epochs
                                                                         ,adamLearningRates,L2Rates,trainScores,devScores)
        modelParametersDF = modelParametersDF.sort_values(by='trainScore', ascending=False)
        display(modelParametersDF)
        
        top5 = modelParametersDF[0:5]
        
        if len(list(top5['n_NeuronsPerFCLayers'])) > 0:
            bestTrainScoreOfSet = list(top5['trainScore'])[0]
            if bestTrainScoreOfSet > 0.01:
                possibleNumberOfNeuronsPerFCLayer = basicUtil.getAdjustedRange(top5['n_NeuronsPerFCLayers'])
        
                possibleNumberOfEpochs = basicUtil.getAdjustedRange(top5['n_Epochs'])
                adamLearningRateCriticalPoints = basicUtil.calculateLogisticCriticalPoints(top5['adamLearningRates'])
                L2CriticalPoints = basicUtil.calculateLogisticCriticalPoints(top5['L2Rates'])
            else:
                possibleNumberOfNeuronsPerFCLayer = increaseIntegerRange(possibleNumberOfNeuronsPerFCLayer)
                possibleNumberOfEpochs = increaseIntegerRange(possibleNumberOfEpochs)
                adamLearningRateCriticalPoints = increaseFloatRange(adamLearningRateCriticalPoints)
                L2CriticalPoints = increaseFloatRange(L2CriticalPoints)

        n_NeuronsPerFCLayers = []
        n_Epochs = []
        adamLearningRates = []
        L2Rates = []
        trainScores = []
        devScores = []

        if bestDevScore > 0.91:
            trial = 101
            
basicUtil.displayFinalResults('../Models/best_effecientnet_model_params.json')
    

In non split
Found 2392 files belonging to 2 classes.
In split
Found 597 files belonging to 2 classes.
Using 299 files for training.
Using 298 files for validation.


2024-09-27 17:47:31.698720: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-09-27 17:47:31.698862: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-09-27 17:47:33.995004: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


trainScore
75/75 [==============================] - 21s 258ms/step - loss: 66.5672 - f1_score: 0.0544
devScore


2024-09-27 17:53:10.710520: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [299]
	 [[{{node Placeholder/_4}}]]
2024-09-27 17:53:10.710669: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [299]
	 [[{{node Placeholder/_4}}]]


10/10 [==============================] - 3s 240ms/step - loss: 66.5674 - f1_score: 0.0458


...

,n_NeuronsPerFCLayers,n_Epochs,adamLearningRates,L2Rates,trainScore,devScore
4,8,9,0.000300,41.338540,0.651020,0.640520
7,11,13,0.000300,41.615975,0.650271,0.608087
0,12,9,0.000299,41.227488,0.648814,0.631853
6,12,22,0.000300,41.117618,0.620696,0.624507
5,6,16,0.000300,42.067275,0.518196,0.535679
8,14,11,0.000300,41.766915,0.397381,0.439839
1,10,14,0.000300,41.335503,0.393160,0.361224
9,13,16,0.000300,41.520404,0.313809,0.281388
2,7,24,0.000300,41.528612,0.053601,0.069056
3,9,16,0.000300,41.707516,0.000000,0.000000


trainScore
75/75 [==============================] - 19s 243ms/step - loss: 20.5374 - f1_score: 0.4591
devScore
10/10 [==============================] - 3s 207ms/step - loss: 20.5522 - f1_score: 0.4340
concluding trial  89
trainScore
75/75 [==============================] - 24s 302ms/step - loss: 47.7177 - f1_score: 0.0328
devScore
10/10 [==============================] - 3s 208ms/step - loss: 47.7127 - f1_score: 0.0000e+00
concluding trial  90
trainScore
75/75 [==============================] - 21s 259ms/step - loss: 92.2444 - f1_score: 0.0433
devScore
10/10 [==============================] - 3s 214ms/step - loss: 92.2438 - f1_score: 0.0268
concluding trial  91
trainScore
75/75 [==============================] - 20s 252ms/step - loss: 13.2567 - f1_score: 0.0015
devScore
10/10 [==============================] - 3s 224ms/step - loss: 13.2565 - f1_score: 0.0000e+00
concluding trial  92
trainScore
75/75 [==============================] - 20s 256ms/step - loss: 72.6415 - f1_score: 0.0043
de

,n_NeuronsPerFCLayers,n_Epochs,adamLearningRates,L2Rates,trainScore,devScore
9,11,20,0.000300,42.032796,0.638632,0.628164
6,11,26,0.000300,41.176856,0.565589,0.587502
8,12,23,0.000299,41.333936,0.532267,0.575263
0,6,22,0.000300,41.482559,0.459103,0.434008
7,7,12,0.000300,41.430212,0.169485,0.164562
2,6,9,0.000300,41.918538,0.043327,0.026786
1,12,16,0.000300,41.765280,0.032821,0.000000
4,11,14,0.000300,41.621817,0.004317,0.000000
3,9,24,0.000300,41.882906,0.001481,0.000000
5,6,14,0.000299,41.613714,0.000000,0.000000


trainScore
75/75 [==============================] - 22s 276ms/step - loss: 63.4461 - f1_score: 0.0143
devScore
10/10 [==============================] - 3s 226ms/step - loss: 63.4449 - f1_score: 0.0118
concluding trial  99
Final Model


,n_NeuronsPerFCLayers,n_Epochs,adamLearningRates,L2Rates,modelSize,trainScore,devScore,testScore
0,11,17,0.000471,25.279694,32.787434,0.632845,0.658997,0.669926
